# Azure AI Agents with Functions - Complete Tutorial

🔧 **Supercharge your Azure AI Agents with Functions!**

This tutorial builds on the basic Azure AI Agents concepts and shows you how to give your agents **superpowers** by letting them use functions!

## What You'll Learn:

1. **Function Basics** - What are functions and why use them?
2. **Simple Functions** - Creating your first agent function
3. **Real-World Examples** - Weather, calculations, and data processing
4. **Advanced Functions** - Multiple parameters and complex logic
5. **Best Practices** - Error handling and function design

**Prerequisites:** You should have completed the basic Azure AI Agents tutorial first!

---

## 🛠️ What are Agent Functions?

Imagine your AI agent as a **smart assistant**, but one that can actually **DO things** beyond just chatting!

### Without Functions:
- Agent can only provide text responses
- Limited to knowledge from training data
- Can't access real-time information
- Can't perform calculations or operations

### With Functions:
- Agent can call external APIs 🌐
- Perform calculations and data processing 🧮
- Access databases and files 📁
- Interact with other services 🔗
- Take actions in the real world! 🚀

### Key Concepts:

- **Function Definition**: You write Python functions with specific signatures
- **Function Registration**: You tell the agent about these functions
- **Function Calling**: The agent decides when and how to use them
- **Function Results**: The agent uses the results to provide better responses

Let's see this in action! ⚡

## 📋 Setup and Prerequisites

Before we start, make sure you have:

1. ✅ Completed the basic Azure AI Agents tutorial
2. ✅ Environment variables set (`PROJECT_ENDPOINT`, `MODEL_DEPLOYMENT_NAME`)
3. ✅ Azure authentication configured

We'll install any additional packages we need and import everything required for function-enabled agents.

In [16]:
# Install required packages
# !pip install azure-ai-agents azure-identity requests

# Import everything we need
import os
import time
import json
import requests
import random
from datetime import datetime
from typing import Any, Dict
from typing import Any, Callable, Set, Dict, List, Optional

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import (
    FunctionTool,
    ToolSet,
    CodeInterpreterTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
    ListSortOrder
)
from azure.identity import DefaultAzureCredential

print("📦 All packages imported successfully!")
print("🔧 Ready to create function-enabled agents!")

# Verify environment variables
required_vars = ['PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME']
for var in required_vars:
    if var in os.environ:
        print(f"✅ {var} is set")
    else:
        print(f"❌ {var} is missing!")

📦 All packages imported successfully!
🔧 Ready to create function-enabled agents!
✅ PROJECT_ENDPOINT is set
✅ MODEL_DEPLOYMENT_NAME is set


In [24]:
# Create our Azure AI Agents client
def create_agents_client():
    try:
        agents_client = AgentsClient(
            endpoint=os.environ["PROJECT_ENDPOINT"],
            credential=DefaultAzureCredential()
        )
        print("🎉 Successfully connected to Azure AI Agents!")
        return agents_client
    except Exception as e:
        print(f"❌ Error creating client: {e}")
        return None

# Create the client
agents_client = create_agents_client()

🎉 Successfully connected to Azure AI Agents!


## 🎯 Your First Function: Simple Calculator

Let's start with something simple - a calculator function that your agent can use to perform math operations.

**What we'll do:**
1. Write a Python function that adds two numbers
2. Create a function definition that describes it to the agent
3. Register the function with our agent
4. Test it out!

**The magic:** The agent will automatically decide when to use this function based on user requests!

In [6]:
# Step 1: Write our Python function
def add_numbers(a: float, b: float) -> float:
    """
    Adds two numbers together.
    
    Args:
        a (float): First number
        b (float): Second number
        
    Returns:
        float: The sum of a and b
    """
    result = a + b
    print(f"🧮 Calculator called: {a} + {b} = {result}")
    return result

# Step 2: Create a FunctionTool with our function
# The FunctionTool automatically handles the function registration
calculator_functions = FunctionTool(functions=[add_numbers])

print("✅ Calculator function ready!")
print(f"📝 Function definitions: {len(calculator_functions.definitions)} tool(s) available")
print(f"📝 Function name: add_numbers")
print(f"📋 Description: Adds two numbers together")

✅ Calculator function ready!
📝 Function definitions: 1 tool(s) available
📝 Function name: add_numbers
📋 Description: Adds two numbers together


In [10]:
# Step 4: Create an agent with our function
calculator_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="math-genius",
    instructions="You are a helpful math assistant. When users ask for calculations, use the available functions to compute accurate results. Always show your work and explain the calculation.",
    tools=calculator_functions.definitions  # Pass the function definitions
)

print(f"🤖 Created calculator agent: {calculator_agent.name}")
print(f"🛠️ Agent has {len(calculator_agent.tools)} tool(s) available")
print(f"🆔 Agent ID: {calculator_agent.id}")

🤖 Created calculator agent: math-genius
🛠️ Agent has 1 tool(s) available
🆔 Agent ID: asst_lUqEbQB4JU2z7e3wisH6WyLS


## 🚀 Testing Our Calculator Agent

Now let's test our function-enabled agent! We'll ask it to do some math and watch it automatically use our function.

**What to expect:**
1. We send a math question
2. The agent recognizes it needs to calculate something
3. The agent calls our `add_numbers` function
4. We provide the function result back to the agent
5. The agent incorporates the result into its response

**Note:** Function calling requires a bit more handling than basic conversations!

In [11]:
# Function to handle tool calls
def handle_function_call(function_call):
    """
    Handles function calls from the agent.
    """
    function_name = function_call.name
    function_args = json.loads(function_call.arguments)
    
    print(f"🔧 Agent wants to call function: {function_name}")
    print(f"📥 With arguments: {function_args}")
    
    # Route to the appropriate function
    if function_name == "add_numbers":
        result = add_numbers(
            a=function_args.get("a"),
            b=function_args.get("b")
        )
        return str(result)
    else:
        return f"Unknown function: {function_name}"


# Create a thread and send a math question
thread = agents_client.threads.create()

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hi! Can you help me calculate 125 + 387? Please show me the exact result."
)

print(f"💬 Sent message: {message.content[0].text.value}")

# Create a run
run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=calculator_agent.id
)

print(f"🏃‍♂️ Started run: {run.id}")

# Poll and handle function calls
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"📊 Run status: {run.status}")
    
    # Handle function calls
    if run.status == "requires_action":
        print("⚡ Agent needs to call a function!")
        
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        
        for tool_call in tool_calls:
            if tool_call.type == "function":
                result = handle_function_call(tool_call.function)
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": result
                })
        
        # Submit the function results back to the agent
        run = agents_client.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
        print("✅ Function results submitted to agent")

print(f"🎉 Run completed with status: {run.status}")

💬 Sent message: Hi! Can you help me calculate 125 + 387? Please show me the exact result.
🏃‍♂️ Started run: run_46EPZLRt27YyZrtHgBQaa59M
📊 Run status: RunStatus.REQUIRES_ACTION
⚡ Agent needs to call a function!
🔧 Agent wants to call function: add_numbers
📥 With arguments: {'a': 125, 'b': 387}
🧮 Calculator called: 125 + 387 = 512
✅ Function results submitted to agent
📊 Run status: RunStatus.COMPLETED
🎉 Run completed with status: RunStatus.COMPLETED


In [12]:
# Display the conversation
messages = agents_client.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.ASCENDING
)

print(f"\n💬 Complete Conversation:")
print(f"═" * 60)

for i, msg in enumerate(messages, 1):
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        role_emoji = "👤" if msg.role == "user" else "🤖"
        print(f"\n{i}. {role_emoji} {msg.role.upper()}:")
        print(f"   {last_text.text.value}")

print(f"\n" + "═" * 60)
print(f"🎯 Amazing! The agent used our function to provide an accurate calculation!")

# Cleanup
agents_client.delete_agent(calculator_agent.id)
print(f"\n🧹 Calculator agent deleted.")


💬 Complete Conversation:
════════════════════════════════════════════════════════════

1. 👤 USER:
   Hi! Can you help me calculate 125 + 387? Please show me the exact result.

2. 🤖 ASSISTANT:
   Sure! To find 125 + 387, you simply add the two numbers together:

125 + 387 = 512

So, the exact result is 512.

════════════════════════════════════════════════════════════
🎯 Amazing! The agent used our function to provide an accurate calculation!

🧹 Calculator agent deleted.


---

## 🌟 Level Up: Real-World Functions

Now let's create more practical functions that agents can use in real scenarios:

1. **Weather Function** - Get current weather information
2. **Random Generator** - Generate random numbers or choices
3. **Date/Time Function** - Get current date and time information
4. **Text Processing** - Analyze and manipulate text

These examples show how functions can make your agents incredibly powerful!

This time we will use the `ToolSet`, which is a collection of tools that can be used by an synchronize agent.

In [21]:
# Multiple useful functions for our agent

# 1. Weather function (mock implementation)
def get_weather(city: str) -> str:
    """
    Gets the current weather for a city (mock implementation).
    """
    # In a real implementation, you'd call a weather API
    weather_options = [
        f"Sunny and 72°F in {city}",
        f"Partly cloudy and 68°F in {city}", 
        f"Rainy and 61°F in {city}",
        f"Overcast and 65°F in {city}"
    ]
    weather = random.choice(weather_options)
    print(f"🌤️ Weather check: {weather}")
    return weather

# 2. Random number generator
def generate_random_number(min_val: int, max_val: int) -> int:
    """
    Generates a random number between min and max values.
    """
    result = random.randint(min_val, max_val)
    print(f"🎲 Generated random number: {result} (between {min_val} and {max_val})")
    return result

# 3. Current date/time function
def get_current_datetime() -> str:
    """
    Gets the current date and time.
    """
    now = datetime.now()
    formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
    print(f"🕐 Current time: {formatted_time}")
    return formatted_time

# 4. Text analysis function
def analyze_text(text: str) -> Dict[str, Any]:
    """
    Analyzes text and returns statistics.
    """
    words = text.split()
    analysis = {
        "character_count": len(text),
        "word_count": len(words),
        "sentence_count": text.count('.') + text.count('!') + text.count('?'),
        "average_word_length": round(sum(len(word) for word in words) / len(words), 2) if words else 0
    }
    print(f"📊 Text analysis complete: {analysis}")
    return analysis

print("✅ All utility functions defined!")

✅ All utility functions defined!


In [20]:
# Create function definitions for the agent
user_functions: Set[Callable[..., Any]] = {
    get_weather,
    generate_random_number,
    get_current_datetime,
    analyze_text
}

# Create tool definitions
toolset = ToolSet()

toolset.add(FunctionTool(functions=user_functions))

# Create a list of tools
for i, tool in enumerate(toolset.definitions, 1):
    print(f"  {i}. {tool.function.name} - {tool.function.description}")

  1. generate_random_number - Generates a random number between min and max values.
  2. get_weather - Gets the current weather for a city (mock implementation).
  3. get_current_datetime - Gets the current date and time.
  4. analyze_text - Analyzes text and returns statistics.


In [25]:
# Create a super-powered agent with multiple functions
super_agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="super-assistant",
    instructions="""You are a versatile AI assistant with access to multiple tools. You can:
    - Get weather information for any city
    - Generate random numbers
    - Provide current date and time
    - Analyze text statistics
    
    Always use the appropriate tools when users request these capabilities. 
    Be helpful, accurate, and explain what you're doing when you use functions.""",
    toolset=toolset,
)

print(f"🚀 Created super agent: {super_agent.name}")
print(f"🛠️ Agent has {len(super_agent.tools)} tools available:")
for i, tool in enumerate(super_agent.tools, 1):
    print(f"  {i}. {tool.function.name}")

🚀 Created super agent: super-assistant
🛠️ Agent has 4 tools available:
  1. generate_random_number
  2. get_weather
  3. get_current_datetime
  4. analyze_text


In [26]:
# Enhanced function handler for multiple functions
def handle_multi_function_call(function_call):
    """
    Handles multiple types of function calls from the agent.
    """
    function_name = function_call.name
    function_args = json.loads(function_call.arguments)
    
    print(f"🔧 Agent calling: {function_name}")
    print(f"📥 Arguments: {function_args}")
    
    try:
        if function_name == "get_weather":
            result = get_weather(city=function_args.get("city"))
            return result
            
        elif function_name == "generate_random_number":
            result = generate_random_number(
                min_val=function_args.get("min_val"),
                max_val=function_args.get("max_val")
            )
            return str(result)
            
        elif function_name == "get_current_datetime":
            result = get_current_datetime()
            return result
            
        elif function_name == "analyze_text":
            result = analyze_text(text=function_args.get("text"))
            return json.dumps(result)
            
        else:
            return f"Unknown function: {function_name}"
            
    except Exception as e:
        print(f"❌ Error executing function {function_name}: {e}")
        return f"Error: {str(e)}"

print("✅ Multi-function handler ready!")

✅ Multi-function handler ready!


In [27]:
# Test the super agent with multiple function calls

# Create a new conversation
thread = agents_client.threads.create()

# Send a complex request that might use multiple functions
complex_message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="""Hi! I need help with a few things:
    1. What's the weather like in Seattle?
    2. What's the current date and time?
    3. Can you generate a random number between 1 and 100?
    4. Please analyze this text: 'Azure AI Agents are incredibly powerful tools for building intelligent applications!'
    
    Thanks!"""
)

print(f"💬 Sent complex request with multiple tasks")

# Create and monitor the run
run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=super_agent.id
)

function_calls_made = 0

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    
    if run.status == "requires_action":
        print(f"⚡ Agent needs to call function(s)! (Call #{function_calls_made + 1})")
        
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        
        for tool_call in tool_calls:
            if tool_call.type == "function":
                result = handle_multi_function_call(tool_call.function)
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": result
                })
                function_calls_made += 1
        
        # Submit all function results
        run = agents_client.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
        print(f"✅ Submitted {len(tool_outputs)} function result(s)")
    
    else:
        print(f"📊 Run status: {run.status}")

print(f"🎉 Run completed! Agent made {function_calls_made} function calls total.")

💬 Sent complex request with multiple tasks
⚡ Agent needs to call function(s)! (Call #1)
🔧 Agent calling: get_weather
📥 Arguments: {'city': 'Seattle'}
🌤️ Weather check: Overcast and 65°F in Seattle
🔧 Agent calling: get_current_datetime
📥 Arguments: {}
🕐 Current time: 2025-06-02 14:39:49
🔧 Agent calling: generate_random_number
📥 Arguments: {'min_val': 1, 'max_val': 100}
🎲 Generated random number: 24 (between 1 and 100)
🔧 Agent calling: analyze_text
📥 Arguments: {'text': 'Azure AI Agents are incredibly powerful tools for building intelligent applications!'}
📊 Text analysis complete: {'character_count': 84, 'word_count': 11, 'sentence_count': 1, 'average_word_length': 6.73}
✅ Submitted 4 function result(s)
📊 Run status: RunStatus.COMPLETED
🎉 Run completed! Agent made 4 function calls total.


In [28]:
# Display the amazing conversation
messages = agents_client.messages.list(
    thread_id=thread.id,
    order=ListSortOrder.ASCENDING
)

print(f"\n💬 Super Agent Conversation:")
print(f"═" * 80)

for i, msg in enumerate(messages, 1):
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        role_emoji = "👤" if msg.role == "user" else "🤖"
        print(f"\n{i}. {role_emoji} {msg.role.upper()}:")
        
        # Split long messages for better readability
        content = last_text.text.value
        if len(content) > 500:
            lines = content.split('\n')
            for line in lines:
                if line.strip():
                    print(f"   {line}")
        else:
            print(f"   {content}")

print(f"\n" + "═" * 80)
print(f"🌟 Incredible! The agent intelligently used multiple functions to complete all tasks!")

# Cleanup
agents_client.delete_agent(super_agent.id)
print(f"\n🧹 Super agent deleted.")


💬 Super Agent Conversation:
════════════════════════════════════════════════════════════════════════════════

1. 👤 USER:
   Hi! I need help with a few things:
    1. What's the weather like in Seattle?
    2. What's the current date and time?
    3. Can you generate a random number between 1 and 100?
    4. Please analyze this text: 'Azure AI Agents are incredibly powerful tools for building intelligent applications!'

    Thanks!

2. 🤖 ASSISTANT:
   Here are the answers to your requests:

1. The current weather in Seattle is overcast and 65°F.
2. The current date and time is 2025-06-02 14:39:49.
3. A random number between 1 and 100: 24.
4. Text analysis of your sentence:
   - Character count: 84
   - Word count: 11
   - Sentence count: 1
   - Average word length: 6.73

Let me know if you need anything else!

════════════════════════════════════════════════════════════════════════════════
🌟 Incredible! The agent intelligently used multiple functions to complete all tasks!

🧹 Super age

---

## 🎯 Practice Exercise: Build Your Own Function

Now it's your turn! Let's create a custom function and add it to an agent.

**Your Mission:**
1. Create a function that does something useful
2. Write the function definition for the agent
3. Test it with a custom agent

**Function Ideas:**
- **Password Generator**: Generate secure passwords
- **Unit Converter**: Convert between different units (temperature, distance, etc.)
- **Word Game**: Create word puzzles or rhymes
- **Simple Calculator**: More math operations (multiply, divide, square root)
- **Color Palette**: Generate color combinations

**Exercise template below - customize it!**

In [30]:
# 🎯 YOUR TURN! Create your own function

# Example: Password Generator Function
def generate_password(length: int = 12, include_symbols: bool = True) -> str:
    """
    Generates a secure password.
    
    Args:
        length (int): Length of the password (default 12)
        include_symbols (bool): Whether to include symbols (default True)
        
    Returns:
        str: Generated password
    """
    import string
    
    characters = string.ascii_letters + string.digits
    if include_symbols:
        characters += "!@#$%^&*"
    
    password = ''.join(random.choice(characters) for _ in range(length))
    print(f"🔐 Generated password of length {length}")
    return password

# TODO: Replace this with your own function!
# def your_custom_function(param1, param2):
#     """Your function description"""
#     # Your logic here
#     return result

# Create function definition
# Initialize function tool with user functions
custom_functions = FunctionTool(functions=[generate_password])

# TODO: Create your own function definition here!

print("✅ Custom function ready for testing!")

✅ Custom function ready for testing!


In [31]:
# Create the client
agents_client = create_agents_client()

# Test your custom function
with agents_client:
    # Create agent with your custom function
    custom_agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="custom-assistant",
        instructions="You are a helpful assistant with custom capabilities. Use your functions when users request them. Be friendly and explain what you're doing.",
        tools=custom_functions.definitions  # Pass the function definitions
    )
    
    print(f"🎨 Created custom agent: {custom_agent.name}")
    
    # Test conversation
    thread = agents_client.threads.create()
    
    # TODO: Customize this message to test your function!
    test_message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="Hi! Can you generate a secure password for me? Make it 16 characters long and include symbols."
    )
    
    print(f"💬 Test message: {test_message.content[0].text.value}")
    
    # Run with function handling
    run = agents_client.runs.create(
        thread_id=thread.id,
        agent_id=custom_agent.id
    )
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
        
        if run.status == "requires_action":
            print("⚡ Testing your custom function!")
            
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_outputs = []
            
            for tool_call in tool_calls:
                if tool_call.type == "function":
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    # Handle your custom function
                    if function_name == "generate_password":
                        result = generate_password(
                            length=function_args.get("length", 12),
                            include_symbols=function_args.get("include_symbols", True)
                        )
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "output": result
                        })
                    
                    # TODO: Add handling for your custom function here!
            
            run = agents_client.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs
            )
    
    # Display results
    messages = agents_client.messages.list(
        thread_id=thread.id,
        order=ListSortOrder.ASCENDING
    )
    
    print(f"\n💬 Custom Function Test Results:")
    print(f"─" * 50)
    
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            role_emoji = "👤" if msg.role == "user" else "🤖"
            print(f"\n{role_emoji} {msg.role.upper()}:")
            print(f"{last_text.text.value}")
    
    print(f"\n🎉 Great job! Your custom function worked perfectly!")
    
    # Cleanup
    agents_client.delete_agent(custom_agent.id)
    print(f"\n🧹 Custom agent deleted.")

🎉 Successfully connected to Azure AI Agents!
🎨 Created custom agent: custom-assistant
💬 Test message: Hi! Can you generate a secure password for me? Make it 16 characters long and include symbols.
⚡ Testing your custom function!
🔐 Generated password of length 16

💬 Custom Function Test Results:
──────────────────────────────────────────────────

👤 USER:
Hi! Can you generate a secure password for me? Make it 16 characters long and include symbols.

🤖 ASSISTANT:
Here is a secure, 16-character password that includes symbols: @OqL%PiI7V38wfk#

Let me know if you'd like another password or if you need it customized in a different way!

🎉 Great job! Your custom function worked perfectly!

🧹 Custom agent deleted.


---

## 🏆 Best Practices for Agent Functions

### ✅ Function Design:

1. **Clear Names**: Use descriptive function names (`get_weather` not `gw`)
2. **Good Descriptions**: Write clear descriptions for both function and parameters
3. **Proper Types**: Use correct parameter types (string, number, boolean, etc.)
4. **Error Handling**: Always handle potential errors gracefully
5. **Return Values**: Return appropriate data types and formats

### ✅ Security Considerations:

1. **Input Validation**: Always validate function parameters
2. **API Keys**: Never hardcode sensitive information
3. **Rate Limiting**: Be mindful of API rate limits
4. **Data Privacy**: Don't log sensitive user data

### ✅ Performance Tips:

1. **Fast Functions**: Keep functions quick to maintain good user experience
2. **Caching**: Cache results when appropriate
3. **Async Operations**: Consider async functions for I/O operations
4. **Error Messages**: Provide helpful error messages

### ✅ Testing:

1. **Unit Tests**: Test your functions independently
2. **Edge Cases**: Test with various input combinations
3. **Error Scenarios**: Test how functions handle errors
4. **Integration**: Test the full agent + function workflow

## 🔧 Troubleshooting Common Issues

### Function Not Called
**Problem**: Agent doesn't use your function
**Solutions**: 
- Check function description clarity
- Verify parameter requirements
- Make sure the user request matches function capability
- Test with more explicit requests

### JSON Parse Errors
**Problem**: `json.loads()` fails on function arguments
**Solutions**:
- Add try/catch around JSON parsing
- Log the raw arguments to debug
- Check parameter type definitions

### Function Execution Errors
**Problem**: Function throws exceptions
**Solutions**:
- Add proper error handling in functions
- Validate all input parameters
- Return error messages instead of throwing exceptions
- Log errors for debugging

### Agent Gets Stuck
**Problem**: Run stays in "requires_action" status
**Solutions**:
- Ensure all tool_call_ids are included in outputs
- Check that function results are returned as strings
- Verify tool_outputs format is correct

### Performance Issues
**Problem**: Functions are slow
**Solutions**:
- Optimize function logic
- Add timeouts to external API calls
- Cache frequently requested data
- Consider async implementations for I/O operations

---

## 🎓 Congratulations! You're Now a Function Master!

### What You've Accomplished:

✅ **Function Fundamentals**:
- Understanding how agent functions work
- Creating function definitions and tool registrations
- Handling function calls and results

✅ **Real-World Examples**:
- Built calculator, weather, random number, and text analysis functions
- Created multi-function agents
- Handled complex conversations with multiple function calls

✅ **Best Practices**:
- Learned proper function design principles
- Understood security and performance considerations
- Mastered error handling and troubleshooting

### 🚀 Next Level Capabilities:

Now you can create agents that:
- 🌐 **Access External APIs** (weather, news, databases)
- 🧮 **Perform Complex Calculations** (scientific, financial)
- 📊 **Process Data** (analyze files, generate reports)
- 🎮 **Create Interactive Experiences** (games, quizzes)
- 🤖 **Automate Tasks** (send emails, manage calendars)

### 🔮 Advanced Topics to Explore:

1. **Async Functions**: For better performance with I/O operations
2. **Function Chaining**: Have functions call other functions
3. **State Management**: Maintain data across function calls
4. **External Integrations**: Connect to databases, APIs, and services
5. **Function Libraries**: Build reusable function collections
6. **Production Deployment**: Scale function-enabled agents

### 💡 Key Takeaways:

- **Functions transform agents from chatbots into action-takers**
- **Good function design makes agents more intelligent and useful**
- **Error handling and testing are crucial for reliable agents**
- **The possibilities are endless with creative function combinations**

**Keep building amazing function-powered agents!** 🌟

---

*Remember: With great function power comes great responsibility! Always consider security, privacy, and user experience when designing agent functions.*

## 📚 Quick Reference

### Function Definition Template:
```python
def my_function(param1: type, param2: type) -> return_type:
    """Clear description of what the function does."""
    # Your logic here
    return result

function_def = FunctionDefinition(
    name="my_function",
    description="Clear description for the agent",
    parameters={
        "type": "object",
        "properties": {
            "param1": {
                "type": "string",  # or "number", "boolean", "array", "object"
                "description": "What this parameter does"
            }
        },
        "required": ["param1"]
    }
)
```

### Function Handler Template:
```python
def handle_function_call(function_call):
    function_name = function_call.name
    function_args = json.loads(function_call.arguments)
    
    if function_name == "my_function":
        result = my_function(**function_args)
        return str(result)
    else:
        return f"Unknown function: {function_name}"
```

### Run with Functions Template:
```python
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    
    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        
        for tool_call in tool_calls:
            if tool_call.type == "function":
                result = handle_function_call(tool_call.function)
                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": result
                })
        
        run = agents_client.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
```

## 🔗 Additional Resources

### Documentation:
- [Azure AI Agents Documentation](https://docs.microsoft.com/azure/ai-services/)
- [Function Calling Best Practices](https://docs.microsoft.com/azure/ai-services/)
- [JSON Schema Reference](https://json-schema.org/)

### Examples to Try:
- **File Processing**: Functions that read/write files
- **Database Operations**: Connect to databases
- **Image Processing**: Analyze or manipulate images
- **Email/SMS**: Send notifications
- **Web Scraping**: Get data from websites
- **Machine Learning**: Run ML models as functions

### Community:
- Share your functions with the community
- Learn from other developers' implementations
- Contribute to open-source function libraries

**Happy coding with Azure AI Agents Functions!** 🎉